## Python Driver Example
### (1) Driven by code

In [ ]:
import sys
sys.path.append("/Path/To/PostAtmos")
from post_process import Post_Process;
extracted_layers = [1] # # Confirm the modification of altitude layers. The first layer is mandatory, with the format like [1,5,10]
sites_includedprovinces = ['all'] # Confirm the modification, for example, 'Shanghai', 'Jiangsu Province', all need to be in lowercase
sites_includedcities =['all']
# Confirm the modification. The four parameters are processing type (WRF,CMAQ_regularsites,CMAQ_supersites,EMISSION,adjoint_sensitivity,adjoint_semi_normalized_sensitivity),
# start and end dates, data source paths. Multiple paths should be separated by semicolons. Multiple paths can have different grid settings.
objProcess = Post_Process('CMAQ_regularsites','2019-07-01','2019-07-31',\
'/Path/To/Forward_China_12km_2019-06-25_2019-07-31_BAU',\
 extracted_layers,sites_includedprovinces,sites_includedcities);
objProcess.run()

### (2) Driven by GUI Interface

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interactive
import pandas
from IPython.display import display
sys.path.append('/Path/To/PostAtmos');
from post_process import Post_Process;

province_cities = {
    'all': ['all'],
    '北京市': ['北京'],
    '天津市': ['天津'],
    '河北省': ['石家庄', '唐山', '邯郸'],
    '山西省': ['太原', '大同', '运城'],
    '内蒙古自治区': ['呼和浩特', '包头', '乌海'],
    '辽宁省': ['沈阳', '大连', '鞍山'],
    '吉林省': ['长春', '吉林', '四平'],
    '黑龙江省': ['哈尔滨', '齐齐哈尔', '牡丹江'],
    '上海市': ['上海'],
    '江苏省': ['南京', '苏州', '无锡'],
    '浙江省': ['杭州', '宁波', '温州'],
    '安徽省': ['合肥', '芜湖', '马鞍山'],
    '福建省': ['福州', '厦门', '泉州'],
    '江西省': ['南昌', '九江', '赣州'],
    '山东省': ['济南', '青岛', '烟台'],
    '河南省': ['郑州', '洛阳', '开封'],
    '湖北省': ['武汉', '黄石', '襄阳'],
    '湖南省': ['长沙', '株洲', '湘潭'],
    '广东省': ['广州', '深圳', '珠海'],
    '广西壮族自治区': ['南宁', '桂林', '柳州'],
    '海南省': ['海口', '三亚', '三沙'],
    '重庆市': ['重庆'],
    '四川省': ['成都', '绵阳', '德阳'],
    '贵州省': ['贵阳', '遵义', '安顺'],
    '云南省': ['昆明', '曲靖', '玉溪'],
    '西藏自治区': ['拉萨', '日喀则', '昌都'],
    '陕西省': ['西安', '宝鸡', '咸阳'],
    '甘肃省': ['兰州', '天水', '白银'],
    '青海省': ['西宁', '海东', '海西'],
    '宁夏回族自治区': ['银川', '石嘴山', '吴忠'],
    '新疆维吾尔自治区': ['乌鲁木齐', '克拉玛依', '吐鲁番']
}
class PostProcessGUI:
    def __init__(self):
        self.objProcess = None

        # Creating widgets for input parameters
        self.model_select = widgets.Dropdown(
            options=['CMAQ_regularsites', 'CMAQ_supersites', 'WRF', 'EMISSION', 'adjoint_sensitivity', 'adjoint_semi_normalized_sensitivity'],
            value='CMAQ_regularsites',
            description='Data type：'
        )
        self.start_date = widgets.DatePicker(
            description='Start date：',
            value=pandas.to_datetime('2019-07-01'),
            layout=widgets.Layout(width='23%')
        )

        self.end_date = widgets.DatePicker(
            description='End date：',
            value=pandas.to_datetime('2019-07-31'),
            layout=widgets.Layout(width='23%')
        )
        
        self.file_path = widgets.Text(
            value='/dssg/home/acct-esehazenet/share/temp/chengzhen/Forward_China_12km_2019-06-25_2019-07-31_BAU',
            description='Data Path：',
            layout=widgets.Layout(width='80%')
        )
               
        self.height_layers = widgets.SelectMultiple(
            options=list(range(1, 45)),
            value=[1],
            description='Layers：'
        )

        self.provinces_select = widgets.Dropdown(
            options=list(province_cities.keys()),
            value='all',
            description='Province：',
            layout=widgets.Layout(width='20%')
        )

        self.cities_select = widgets.Dropdown(
            options=['all'],
            value='all',
            description='City：',
            layout=widgets.Layout(width='20%')
        )

        self.blank_label = widgets.Label()
        self.status_label = widgets.Label()

        # Creating a button to trigger the process
        self.run_button = widgets.Button(description='RUN', button_style='success')
        self.run_button.on_click(self.run_process)        

        self.date_widgets = widgets.HBox([self.start_date, self.end_date])
        self.region_widgets = widgets.HBox([self.provinces_select, self.cities_select])

        self.layout = widgets.VBox([
            self.model_select,
            self.date_widgets,
            self.file_path,
            self.height_layers,
            self.region_widgets,
            self.blank_label,
            widgets.Box([self.run_button, widgets.Label()], layout=widgets.Layout(justify_content='center'))
        ])

        self.provinces_select.observe(self.on_province_change, names='value')

    def on_province_change(self, change):
        selected_province = change['new']
        if selected_province == 'all':
            self.cities_select.options = ['all']
        else:
            self.cities_select.options = ['all'] + province_cities[selected_province]

    def run_process(self, _):
        # Creating an instance of Post_Process and setting parameters
        self.objProcess = Post_Process(
            self.model_select.value,
            str(self.start_date.value).split(' ')[0],
            str(self.end_date.value).split(' ')[0],
            self.file_path.value,
            list(self.height_layers.value),
            [self.provinces_select.value],
            [self.cities_select.value]
        )
        self.objProcess.run()

# Creating an instance of GUI
post_process_gui = PostProcessGUI()

# Displaying the GUI
display(post_process_gui.layout)
